In [22]:
import pandas as pd
import numpy as np

# X Data for Final Stock Selection 2021/2022
Requires SimFin+ Bulk Download
## Get the Latest Share Prices (To make Market Cap. Column Soon)

In [23]:
def getYRawData2022(my_path = 'C:/Users/G50/Stock_Data/SimFin2022/'):
    d=pd.read_csv(my_path + 'us-shareprices-daily.csv', delimiter=';')
    d["Date"]=pd.to_datetime(d["Date"])
    print('Stock Price data matrix is: ',d.shape)
    return d

In [24]:
d=getYRawData2022()

Stock Price data matrix is:  (8252647, 11)


## Functions to Extract Price at Time We Need and Combine X Data

In [25]:
def getYPricesReportDate(X, d, modifier=365):
    '''
    Get the stock prices for our X matrix to create Market Cap. column later.
    '''
    i=0
    y = [[None]*8 for i in range(len(X))] # Preallocation list of list of 2 [(price at date) (price at date + modifier)]
    whichDateCol='Publish Date'# or 'Report Date', is the performance date from->to. Want this to be publish date.
    # Because of time lag between report date (which can't be actioned on) and publish date (data we can trade with)
    for index in range(len(X)):
        y[i]=getYPriceDataNearDate(X['Ticker'].iloc[index], X[whichDateCol].iloc[index], 0, d)
        i=i+1
    return y

In [26]:
def getXFullDataMerged(myLocalPath='C:/Users/G50/Stock_Data/SimFin2022/'):
    '''
    For combining fundamentals financial data from SimFin+ only,
    without API. 
    Download Income Statement, Balance Sheet and Cash Flow files,
    the -full versions, e.g. us-balance-annual-full.csv.
    Place in a directory and give the directory path to the function.
    Assumes standard filenames from SimFin.
    Returns a DataFrame of the combined result. 
    Prints file infos.
    '''
    incomeStatementData=pd.read_csv(myLocalPath+'us-income-annual-full.csv',
                                    delimiter=';')
    balanceSheetData=pd.read_csv(myLocalPath+'us-balance-annual-full.csv',
                                 delimiter=';')
    CashflowData=pd.read_csv(myLocalPath+'us-cashflow-annual-full.csv',
                             delimiter=';')
    
    print('Income Statement CSV data is(rows, columns): ',
          incomeStatementData.shape)
    print('Balance Sheet CSV data is: ',
          balanceSheetData.shape)
    print('Cash Flow CSV data is: ' ,
          CashflowData.shape)
    

    # Merge the data together
    result = pd.merge(incomeStatementData, balanceSheetData,\
                on=['Ticker','SimFinId','Currency',
                    'Fiscal Year','Report Date','Publish Date'])
    
    result = pd.merge(result, CashflowData,\
                on=['Ticker','SimFinId','Currency',
                    'Fiscal Year','Report Date','Publish Date'])
    
    # dates in correct format
    result["Report Date"] = pd.to_datetime(result["Report Date"]) 
    result["Publish Date"] = pd.to_datetime(result["Publish Date"])
    
    print('Merged X data matrix shape is: ', result.shape)
    
    return result

## Combine X Data (2021/22)

In [27]:
X = getXFullDataMerged()
# Get data only for 2022
PublishDateStart = "2021-12-30"
PublishDateEnd = "2022-03-05"
bool_list = X['Publish Date'].between(\
              pd.to_datetime(PublishDateStart),\
              pd.to_datetime(PublishDateEnd) )
X=X[bool_list]

Income Statement CSV data is(rows, columns):  (23652, 69)
Balance Sheet CSV data is:  (23652, 96)
Cash Flow CSV data is:  (23652, 63)
Merged X data matrix shape is:  (23652, 216)


In [28]:
X

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Report Date,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,...,Increase in Capital Stock,Decrease in Capital Stock,Other Financing Activities,Net Cash from Discontinued Operations (Financing),Net Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Change in Cash from Disc. Operations and Other,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Change in Cash
20,AA,367153,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,1.856634e+08,1.856634e+08,...,25000000.0,-150000000.0,-215000000.0,NaN,-1.158000e+09,327000000,NaN,327000000,-13000000.0,314000000
34,AAL,68568,USD,2021,FY,2021-12-31,2022-02-22,2022-02-22,6.487660e+08,6.487660e+08,...,442000000.0,NaN,-1000000.0,NaN,5.288000e+09,9000000,NaN,9000000,NaN,9000000
55,AAN,441241,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,3.248500e+07,3.318800e+07,...,2587000.0,-103098000.0,-2729000.0,NaN,-1.039510e+08,-53286000,NaN,-53286000,-5000.0,-53291000
65,AAOI,671827,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,2.709737e+07,2.709737e+07,...,15405000.0,NaN,-978000.0,NaN,1.408800e+07,-8102000,NaN,-8102000,-876000.0,-8978000
77,AAON,919501,USD,2021,FY,2021-12-31,2022-02-28,2022-02-28,5.242071e+07,5.354651e+07,...,21148000.0,-20876000.0,-1590000.0,NaN,1.873500e+07,-78801000,NaN,-78801000,NaN,-78801000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23561,ZGNX,816789,USD,2021,FY,2021-12-31,2022-03-01,2022-03-01,5.590500e+07,5.590500e+07,...,1962000.0,NaN,-19788000.0,NaN,-1.782600e+07,-65736000,NaN,-65736000,NaN,-65736000
23564,ZI,1205635,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,2.632858e+08,2.639996e+08,...,2100000.0,NaN,-53000000.0,NaN,4.395000e+08,43100001,NaN,43100001,NaN,43100001
23591,ZNGA,418866,USD,2021,FY,2021-12-31,2022-02-25,2022-02-25,1.097100e+09,1.097100e+09,...,33400000.0,NaN,-73600000.0,NaN,-4.140000e+07,-288500000,NaN,-288500000,-26700000.0,-315200000
23607,ZTS,378251,USD,2021,FY,2021-12-31,2022-02-15,2022-02-15,4.740000e+08,4.763000e+08,...,NaN,NaN,-41000000.0,NaN,-1.862000e+09,-107000000,NaN,-107000000,-12000000.0,-119000000


## Combine Y Data (2020/21)

In [29]:
def getYPriceDataNearDate(ticker, date, modifier, dailySharePrices):
    '''
    Return just the y price and volume.
    Take the first day price/volume of the list of days,
    that fall in the window of accepted days.
    'modifier' just modifies the date to look between.
    Returns a list.
    '''
    windowDays=5
    rows = dailySharePrices[
        (dailySharePrices['Date'].between(pd.to_datetime(date)
                                          + pd.Timedelta(days=modifier),
                                          pd.to_datetime(date)
                                          + pd.Timedelta(days=windowDays
                                                         +modifier)
                                         )
        ) & (dailySharePrices['Ticker']==ticker)]
    
    if rows.empty:
        return [ticker, np.float('NaN'),\
                np.datetime64('NaT'),\
                np.float('NaN')]
    else:
        return [ticker, rows.iloc[0]['Open'],\
                rows.iloc[0]['Date'],\
                rows.iloc[0]['Volume']*rows.iloc[0]['Open']]

In [30]:
y = getYPricesReportDate(X, d) # Takes a min due to price lookups.
y = pd.DataFrame(y, columns=['Ticker', 'Open Price', 'Date', 'Volume'])

C:\Users\G50\AppData\Local\Temp\ipykernel_8024\3741595865.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return [ticker, np.float('NaN'),\
C:\Users\G50\AppData\Local\Temp\ipykernel_8024\3741595865.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.float('NaN')]


In [32]:
y

,Ticker,Open Price,Date,Volume
0,AA,75.29,2022-02-24,8.741855e+08
1,AAL,17.44,2022-02-22,5.622818e+08
2,AAN,19.42,2022-02-24,1.523043e+07
3,AAOI,3.74,2022-02-24,2.858437e+06
4,AAON,57.88,2022-02-28,1.801839e+07
...,...,...,...,...
884,ZGNX,26.23,2022-03-01,6.336510e+07
885,ZI,45.92,2022-02-24,1.780396e+08
886,ZNGA,9.10,2022-02-25,5.199268e+08
887,ZTS,197.79,2022-02-15,6.726517e+08


## Filter Unwanted Rows with Both X and y, Also Create Market Cap

In [33]:
y=y.reset_index(drop=True)
X=X.reset_index(drop=True)

# Issue where no share price
bool_list1 = ~y["Open Price"].isnull()
# Issue where there is low/no volume
bool_list2 = ~(y['Volume']<1e4)

y=y[bool_list1 & bool_list2]
X=X[bool_list1 & bool_list2]

# Issues where no listed number of shares
bool_list4 = ~X["Shares (Diluted)_x"].isnull()
y=y[bool_list4]
X=X[bool_list4]

# additional filters maybe.
               
y=y.reset_index(drop=True)
X=X.reset_index(drop=True)

X["Market Cap"] = y["Open Price"]*X["Shares (Diluted)_x"]

In [36]:
y

,Ticker,Open Price,Date,Volume
0,AA,75.29,2022-02-24,8.741855e+08
1,AAL,17.44,2022-02-22,5.622818e+08
2,AAN,19.42,2022-02-24,1.523043e+07
3,AAOI,3.74,2022-02-24,2.858437e+06
4,AAON,57.88,2022-02-28,1.801839e+07
...,...,...,...,...
873,ZGNX,26.23,2022-03-01,6.336510e+07
874,ZI,45.92,2022-02-24,1.780396e+08
875,ZNGA,9.10,2022-02-25,5.199268e+08
876,ZTS,197.79,2022-02-15,6.726517e+08


In [37]:
X

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Report Date,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,...,Decrease in Capital Stock,Other Financing Activities,Net Cash from Discontinued Operations (Financing),Net Cash from Financing Activities,Net Cash Before Disc. Operations and FX,Change in Cash from Disc. Operations and Other,Net Cash Before FX,Effect of Foreign Exchange Rates,Net Change in Cash,Market Cap
0,AA,367153,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,1.856634e+08,1.856634e+08,...,-150000000.0,-215000000.0,NaN,-1.158000e+09,327000000,NaN,327000000,-13000000.0,314000000,1.397860e+10
1,AAL,68568,USD,2021,FY,2021-12-31,2022-02-22,2022-02-22,6.487660e+08,6.487660e+08,...,NaN,-1000000.0,NaN,5.288000e+09,9000000,NaN,9000000,NaN,9000000,1.131448e+10
2,AAN,441241,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,3.248500e+07,3.318800e+07,...,-103098000.0,-2729000.0,NaN,-1.039510e+08,-53286000,NaN,-53286000,-5000.0,-53291000,6.445110e+08
3,AAOI,671827,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,2.709737e+07,2.709737e+07,...,NaN,-978000.0,NaN,1.408800e+07,-8102000,NaN,-8102000,-876000.0,-8978000,1.013442e+08
4,AAON,919501,USD,2021,FY,2021-12-31,2022-02-28,2022-02-28,5.242071e+07,5.354651e+07,...,-20876000.0,-1590000.0,NaN,1.873500e+07,-78801000,NaN,-78801000,NaN,-78801000,3.099272e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,ZGNX,816789,USD,2021,FY,2021-12-31,2022-03-01,2022-03-01,5.590500e+07,5.590500e+07,...,NaN,-19788000.0,NaN,-1.782600e+07,-65736000,NaN,-65736000,NaN,-65736000,1.466388e+09
874,ZI,1205635,USD,2021,FY,2021-12-31,2022-02-24,2022-02-24,2.632858e+08,2.639996e+08,...,NaN,-53000000.0,NaN,4.395000e+08,43100001,NaN,43100001,NaN,43100001,1.212286e+10
875,ZNGA,418866,USD,2021,FY,2021-12-31,2022-02-25,2022-02-25,1.097100e+09,1.097100e+09,...,NaN,-73600000.0,NaN,-4.140000e+07,-288500000,NaN,-288500000,-26700000.0,-315200000,9.983610e+09
876,ZTS,378251,USD,2021,FY,2021-12-31,2022-02-15,2022-02-15,4.740000e+08,4.763000e+08,...,NaN,-41000000.0,NaN,-1.862000e+09,-107000000,NaN,-107000000,-12000000.0,-119000000,9.420738e+10


## Save 2021/22 X Data for Stock Selection, y Data for Ticker Names

In [38]:
X.to_csv("Annual_Stock_Price_Fundamentals_Filtered_2022.csv")
y.to_csv("Tickers_Dates_2022.csv")

# X Data for Final Stock Selection 2021/2022
Requires SimFin+ Bulk Download

In [39]:
def addColsToX(x):
    '''
    Takes in x DataFrame, edits it to include:
        Enterprise Value.
        Earnings before interest and tax.
    
    '''
    x['EV'] = x['Market Cap'] \
    + x['Long Term Debt'] \
    + x['Short Term Debt'] \
    - x['Cash, Cash Equivalents & Short Term Investments']

    x['EBIT'] = x['Net Income_x'] \
    - x['Interest Expense, Net'] \
    - x['Income Tax (Expense) Benefit, Net']
    
# Make new X with ratios to learn from.
def getXRatios(x_):
    '''
    Takes in x_, which is the fundamental stock DataFrame raw. 
    Outputs X, which is the data encoded into stock ratios.
    '''
    X=pd.DataFrame()
    
    # EV/EBIT
    X['EV/EBIT'] = x_['EV'] / x_['EBIT']
    
    # Op. In./(NWC+FA)
    X['Op. In./(NWC+FA)'] = x_['Operating Income (Loss)'] \
    / (x_['Total Current Assets'] - x_['Total Current Liabilities'] \
       + x_['Property, Plant & Equipment, Net'])
    
    # P/E
    X['P/E'] = x_['Market Cap'] / x_['Net Income_x']
    
    # P/B
    X['P/B'] = x_['Market Cap'] / x_['Total Equity'] 
    
    # P/S
    X['P/S'] = x_['Market Cap'] / x_['Revenue'] 
    
    # Op. In./Interest Expense
    X['Op. In./Interest Expense'] = x_['Operating Income (Loss)']\
    / - x_['Interest Expense, Net']
    
    # Working Capital Ratio
    X['Working Capital Ratio'] = x_['Total Current Assets']\
    / x_['Total Current Liabilities']
    
    # Return on Equity
    X['RoE'] = x_['Net Income_x'] / x_['Total Equity']
    
    # Return on Capital Employed
    X['ROCE'] = x_['EBIT']\
    / (x_['Total Assets'] - x_['Total Current Liabilities'] )
    
    # Debt/Equity
    X['Debt/Equity'] = x_['Total Liabilities'] / x_['Total Equity']
    
    # Debt Ratio
    X['Debt Ratio'] = x_['Total Assets'] / x_['Total Liabilities']
    
    # Cash Ratio
    X['Cash Ratio'] = x_['Cash, Cash Equivalents & Short Term Investments']\
    / x_['Total Current Liabilities']
    
    # Asset Turnover
    X['Asset Turnover'] = x_['Revenue'] / \
                            x_['Property, Plant & Equipment, Net']
    
    # Gross Profit Margin
    X['Gross Profit Margin'] = x_['Gross Profit'] / x_['Revenue']
    
    ### Altman ratios ###
    # (CA-CL)/TA
    X['(CA-CL)/TA'] = (x_['Total Current Assets']\
                       - x_['Total Current Liabilities'])\
                        /x_['Total Assets']
    
    # RE/TA
    X['RE/TA'] = x_['Retained Earnings']/x_['Total Assets']
    
    # EBIT/TA
    X['EBIT/TA'] = x_['EBIT']/x_['Total Assets']
    
    # Book Equity/TL
    X['Book Equity/TL'] = x_['Total Equity']/x_['Total Liabilities']
    
    X.fillna(0, inplace=True)
    return X

def fixXRatios(X):
    '''
    Takes in X, edits it to have the distributions clipped.
    The distribution clippings are done manually by eye,
    with human judgement based on the information.
    '''
    X['RoE'].clip(-5, 5, inplace=True)
    X['Op. In./(NWC+FA)'].clip(-5, 5, inplace=True)
    X['EV/EBIT'].clip(-500, 500, inplace=True)
    X['P/E'].clip(-1000, 1000, inplace=True)
    X['P/B'].clip(-50, 100, inplace=True)    
    X['P/S'].clip(0, 500, inplace=True)
    X['Op. In./Interest Expense'].clip(-600, 600, inplace=True)#-600, 600
    X['Working Capital Ratio'].clip(0, 30, inplace=True)  
    X['ROCE'].clip(-2, 2, inplace=True)
    X['Debt/Equity'].clip(0, 100, inplace=True)
    X['Debt Ratio'].clip(0, 50, inplace=True)  
    X['Cash Ratio'].clip(0, 30, inplace=True)
    X['Gross Profit Margin'].clip(0, 1, inplace=True) #how can be >100%?
    X['(CA-CL)/TA'].clip(-1.5, 2, inplace=True)
    X['RE/TA'].clip(-20, 2, inplace=True)
    X['EBIT/TA'].clip(-2, 1, inplace=True)
    X['Book Equity/TL'].clip(-2, 20, inplace=True)
    X['Asset Turnover'].clip(-2000, 2000, inplace=True)# 0, 500

In [40]:
X=pd.read_csv("Annual_Stock_Price_Fundamentals_Filtered_2022.csv", 
              index_col=0)

# Net Income fix, checked annual reports.
X['Net Income'] = X['Net Income_x'] 

X.fillna(0)
addColsToX(X)
X=getXRatios(X)
fixXRatios(X)
X.to_csv("Annual_Stock_Price_Fundamentals_Ratios_2022.csv")

In [41]:
# Z score to account for default chance
def calcZScores(X):
    '''
    Calculate Altman Z'' scores 1995
    '''
    Z = pd.DataFrame()
    Z['Z score'] = 3.25 \
    + 6.51 * X['(CA-CL)/TA']\
    + 3.26 * X['RE/TA']\
    + 6.72 * X['EBIT/TA']\
    + 1.05 * X['Book Equity/TL']
    return Z

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import pickle

# Linear model pipeline
def trainLinearModel(X_train, y_train):
    pl_linear = Pipeline([('Power Transformer', PowerTransformer()),
        ('linear', LinearRegression())])
    pl_linear.fit(X_train, y_train)
    return pl_linear

# ElasticNet model pipeline
def trainElasticNetModel(X_train, y_train):
    pl_ElasticNet = Pipeline([('Power Transformer', PowerTransformer()),
        ('ElasticNet', ElasticNet(l1_ratio=0.00001))])
    pl_ElasticNet.fit(X_train, y_train)
    return pl_ElasticNet

# KNeighbors regressor
def trainKNeighborsModel(X_train, y_train):
    pl_KNeighbors = Pipeline([('Power Transformer', PowerTransformer()),
        ('KNeighborsRegressor', KNeighborsRegressor(n_neighbors=40))])
    pl_KNeighbors.fit(X_train, y_train)
    return pl_KNeighbors

# DecisionTreeRegressor
def traindecTreeModel(X_train, y_train):
    pl_decTree = Pipeline([
        ('DecisionTreeRegressor',\
         DecisionTreeRegressor(max_depth=20, random_state=42))
    ])
    pl_decTree.fit(X_train, y_train)
    return pl_decTree

# RandomForestRegressor
def trainrfregressorModel(X_train, y_train):
    pl_rfregressor = Pipeline([
        ('RandomForestRegressor',\
         RandomForestRegressor(max_depth=10, random_state=41))
    ])
    pl_rfregressor.fit(X_train, y_train)
    
    return pl_rfregressor

# GradientBoostingRegressor
def traingbregressorModel(X_train, y_train):
    pl_GradBregressor = Pipeline([
        ('GradBoostRegressor',\
         GradientBoostingRegressor(n_estimators=100,\
                                   learning_rate=0.1,\
                                   max_depth=10,\
                                   random_state=42,\
                                   loss='squared_error'))  ])
    pl_GradBregressor.fit(X_train, y_train)
    
    return pl_GradBregressor

# SVM
def trainsvmModel(X_train, y_train):
    pl_svm = Pipeline([('Power Transformer', PowerTransformer()),
        ('SVR', SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1))])
    pl_svm.fit(X_train, y_train)
    return pl_svm

In [49]:
def pickStockForMe(model='GBoost'):
    '''
    Pick stocks.
    Reads Annual_Stock_Price_Fundamentals_Ratios.csv,
    and Annual_Stock_Price_Performance_Percentage.csv,
    trains the AI with the best model/parameters,
    Then picks stocks using outputs from Notebooks 1 and 2:
    Annual_Stock_Price_Fundamentals_Ratios_2022.csv,
    and Tickers_Dates_2022.csv.
    Outputs a DataFrame of best picks.
    '''
    # Training X and Y of all previous year data
    X=pd.read_csv('Annual_Stock_Price_Fundamentals_Ratios.csv', 
                  index_col=0)
    # annual stock performances
    yperf=pd.read_csv('Annual_Stock_Price_Performance_Percentage.csv', 
                      index_col=0)
    
    yperf=yperf['Perf']

    # Stock selection ratios for 2022 X
    X_2022=pd.read_csv('Annual_Stock_Price_Fundamentals_Ratios_2022.csv', 
                       index_col=0)
    
    # And the row tickers
    tickers=pd.read_csv('Tickers_Dates_2022.csv', index_col=0)
    
    if model == 'GBoost':
        # Gradient Boosted tree
        model_pl = traingbregressorModel(X, yperf)
        y_pred=model_pl.predict(X_2022)
        y_pred=pd.DataFrame(y_pred)
    
    elif model == 'RandForest':
        # 
        model_pl = trainrfregressorModel(X, yperf)
        y_pred=model_pl.predict(X_2022)
        y_pred=pd.DataFrame(y_pred)

    elif model == 'KNeighborsRegressor':
        # KNN
        model_pl = trainKNeighborsModel(X, yperf)
        y_pred=model_pl.predict(X_2022)
        y_pred=pd.DataFrame(y_pred)
    
    else:
        return None

    # FINAL STOCK PICKS
    # Separate out stocks with low Z scores
    # 3.75 is approx. B- rating
    z = calcZScores(X_2022)
    tickers['Z Score'] = z
    tickers['Perf. Score'] = y_pred
    Final_Predictions = pd.DataFrame()
    Final_Predictions = tickers[['Ticker',
                                 'Date',
                                 'Perf. Score',
                                 'Z Score']][tickers['Z Score'] > 3]

    return Final_Predictions.sort_values(by='Perf. Score', 
                                         ascending=False).reset_index(drop=True).head(100)

In [50]:
# Stock Selection
a = pickStockForMe()
a.to_csv('fdsfds.csv')

,Ticker,Date,Perf. Score,Z Score
0,CYCA,2022-01-12,1.775386,15.160241
1,RIDE,2022-02-28,1.756731,5.825899
2,X,2022-02-11,1.504565,7.971893
3,SCPL,2022-03-02,1.351882,10.253768
4,OPEN,2022-02-24,1.331233,6.007376
...,...,...,...,...
95,DOW,2022-02-04,0.284901,6.480732
96,ADBE,2022-01-21,0.283992,9.209643
97,AXGN,2022-02-25,0.281988,4.089196
98,SILK,2022-03-01,0.281214,3.351413


In [47]:
pickStockForMe('RandForest')

,Ticker,Date,Perf. Score,Z Score
0,CYCA,2022-01-12,1.411279,15.160241
1,CPS,2022-02-18,0.931409,3.995975
2,RYAM,2022-03-01,0.631105,5.818901
3,BYND,2022-03-02,0.570243,6.122126
4,HUBS,2022-02-14,0.550189,5.478837
5,OPEN,2022-02-24,0.537823,6.007376
6,SUP,2022-03-03,0.511743,4.440697
7,GEO,2022-02-28,0.507170,4.783551
8,TUSK,2022-03-04,0.498250,6.334484
9,LCID,2022-02-28,0.492233,4.623454


In [48]:
X

,EV/EBIT,Op. In./(NWC+FA),P/E,P/B,P/S,Op. In./Interest Expense,Working Capital Ratio,RoE,ROCE,Debt/Equity,Debt Ratio,Cash Ratio,Asset Turnover,Gross Profit Margin,(CA-CL)/TA,RE/TA,EBIT/TA,Book Equity/TL
0,11.086672,0.246499,32.584150,2.224475,1.150313,10.651282,1.559417,0.068269,0.106168,1.390993,1.718911,0.562830,1.834818,0.246791,0.120000,-0.020965,0.083394,0.718911
1,-48.229085,-0.038002,-5.677109,-1.541482,0.378639,-0.594276,0.912133,0.271526,-0.016140,0.000000,0.900551,0.654057,1.011680,0.068469,-0.025125,-0.129959,-0.011525,-0.099449
2,0.000000,0.318016,5.862708,0.897440,0.349233,157.251370,2.457196,0.153076,0.133416,1.006869,1.993178,0.067756,7.992828,0.628278,0.340701,0.068375,0.102223,0.993178
3,-4.115556,-0.172984,-1.871131,0.398099,0.479021,-10.228108,1.781244,-0.212759,-0.140694,0.785202,2.273557,0.318070,0.870512,0.178267,0.187304,-0.313936,-0.106962,1.273557
4,0.000000,0.177803,52.746386,6.648373,5.798267,524.484848,2.513369,0.126044,0.123025,0.394727,3.533395,0.032950,2.071274,0.257859,0.201963,0.591076,0.106607,2.533395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,0.000000,-0.841486,-6.448128,8.193440,17.950645,-15.202299,3.629373,-1.270670,-0.573608,1.617681,1.618169,3.088368,11.350563,0.940825,0.548303,-3.313318,-0.453994,0.618169
874,0.000000,1.716792,103.791642,6.067803,16.224390,3.120729,1.075059,0.058461,0.026287,2.430052,1.411514,0.643617,17.918465,0.817050,0.005560,0.016460,0.024340,0.411514
875,0.000000,1.061472,-95.811996,3.208204,3.564938,2.321970,1.054497,-0.033484,0.009300,1.043414,1.958392,0.737495,92.425743,0.637600,0.013399,-0.378367,0.007014,0.958392
876,0.000000,0.371013,46.248099,20.732257,12.115146,12.513393,3.856427,0.448283,0.224325,2.058979,1.485678,1.939343,3.210570,0.703832,0.369281,0.516978,0.195324,0.485678
